In [62]:
import os
import pandas as pd
import torch 
import torch.nn.functional as F
from sklearn.cluster import AgglomerativeClustering


num = 0
results = {}
folder_path = '/media/ubuntu/sda/data/mouse11/output/analysis_natural_image/analysis_new/'

csv_files = [f for f in os.listdir(folder_path) if f.startswith('waveform_mean_Neuron_') and f.endswith('.csv')]


for csv_file in csv_files:
    df = pd.read_csv(os.path.join(folder_path, csv_file), index_col=0)
    tensor = torch.tensor(df.values, dtype=torch.float32)
    cosine_similarities = F.cosine_similarity(tensor.unsqueeze(1), tensor.unsqueeze(0), dim=2)
    clustering = AgglomerativeClustering(n_clusters=None, distance_threshold=0.2, linkage='average')
    clustering.fit(1 - cosine_similarities)
    labels = clustering.labels_
    label_counts = pd.Series(labels).value_counts()
    
    valid_labels = label_counts[label_counts >= 16].index
    
    if len(valid_labels) == 0:
        # 如果没有类的数量大于等于16，则将所有数据视为一个类
        results[num] = list(df.index)
        num += 1
    else:
        for valid_label in valid_labels:
            results[num] = [index for index, label in zip(df.index, labels) if label == valid_label]
            num += 1

In [63]:
unique_results = {}
seen_values = set()
for key, value in results.items():
    value_tuple = tuple(value)
    if value_tuple not in seen_values:
        unique_results[key] = value
        seen_values.add(value_tuple)

In [78]:
for key,item in unique_results.items():
    item = set([i.split("_")[0] for i in item])
    if len(item) != 16:
        print(f"{key}: {len(item)}")
        del unique_results[key]